In [1]:
import pandas as pd
import re
import math
import numpy as np
import shapefile
from matplotlib.patches import Polygon

## 1. Calculate the tree canopy cover

In [2]:
tree = pd.read_csv("Trees_df_all.csv")

In [3]:
tree_list = tree.Genus
tree_list=list(set(tree_list))

In [4]:
np.random.seed(123)
bias = {}
for i in tree_list:
    bias[i]=np.random.random()/2

    
tree['Canopy']  = 0.0


for index,age,genus in zip(tree.index,tree.age,tree.Genus):
    if age<=5:
        tree.loc[index,'Canopy']=(1+0.4*age+bias[genus])**2*math.pi
    elif age>5 and age<=10:
        tree.loc[index,'Canopy']=(3+0.4*(age-5)+bias[genus])**2*math.pi
    elif age>10 and age<=23:
        tree.loc[index,'Canopy']=(5+0.13*(age-10)+bias[genus])**2*math.pi
    elif age>23 and age<=35:
        tree.loc[index,'Canopy']=(8+0.1*(age-23)+bias[genus])**2*math.pi
    elif age>35 and age<=65:
        tree.loc[index,'Canopy']=(9+0.2*(age-35)+bias[genus])**2*math.pi
    else:
        tree.loc[index,'Canopy']=(15-0.1*(age-65)+bias[genus])**2*math.pi

In [5]:
tree.to_csv('Trees_df_all_withCanopy.csv',index=False,encoding='UTF-8')

### 2. Calculate the Simpson Index and Species Evenness based on Suburbs

In [29]:
suburb_list = tree.suburb
suburb_set = list(set(suburb_list))

In [30]:
index_genus = pd.DataFrame(columns=['suburb', 'shannon_index', 'simpson_index', 'species_evenness'])
for suburb in suburb_set: 
    temp_dict = {}
    tree_amout = len(tree[tree.suburb == suburb].Genus)
    for item in tree[tree.suburb == suburb].Genus:
        try:
            temp_dict[item] += 1
        except:
            temp_dict[item] = 1
    species_number = 0
    shannon_index = 0
    simpson_temp = 0
    for key in temp_dict.keys():
        simpson_temp += temp_dict[key]*(temp_dict[key]-1)
        shannon_index += -(temp_dict[key]/tree_amout)*math.log(temp_dict[key]/tree_amout)
    
    simpson_index = 1-simpson_temp/(tree_amout*(tree_amout-1))
    
    try:
        species_evenness = shannon_index/math.log(len(temp_dict.keys()))
    except: 
        species_evenness = 0
    
    index_genus = index_genus.append([{'suburb':suburb, 'shannon_index':shannon_index, 'simpson_index':simpson_index, 'species_evenness':species_evenness}], ignore_index=True)

In [8]:
#index_genus.to_csv('tree_index_genus.csv',index=False,encoding='UTF-8')

## 3. Calculate the Suburb Canopy cover rate

In [21]:
sf = shapefile.Reader("./vic_suburb_boundary/VIC_LOCALITY_POLYGON_shp") # note, no suffix, all 3 files are used
recs = sf.records()
shapes = sf.shapes()


In [37]:
index_genus['geo']  = 

In [40]:
for index, sub in zip(index_genus.index,index_genus.suburb):
    for i in range(len(recs)):
        if recs[i].VIC_LOCA_2 == sub:
            index_genus.loc[index,'geo']=str(shapes[i].points)
            
    

In [27]:
#suburb_geo = pd.read_csv("tree_index_genus_geo.csv")

In [42]:
suburb_area={}
for sub,coord in zip(index_genus.suburb,index_genus.geo):
    coord = eval(coord)
    coord_len = len(coord)
    
    s = coord[0][1] * (coord[coord_len -1][0]-coord[1][0])
    for i in range(1,coord_len):
        s += coord[i][1] * (coord[i-1][0] - coord[(i+1)%coord_len][0])
    
    suburb_area[sub] = round(math.fabs(s/2)*9101160000.085981,6)




In [44]:
suburb_canopy = {}
for sub,canopy in zip(tree.suburb,tree.Canopy):
    try:
        suburb_canopy[sub]+=float(canopy)
    except:
        suburb_canopy[sub]=float(canopy)

In [46]:
for index,suburb in zip(index_genus.index,index_genus.suburb):
    index_genus.loc[index,'canopy_rate'] = suburb_canopy[suburb]/suburb_area[suburb]

In [48]:
index_genus.to_csv('tree_suburb_index_with_canopyrate.csv',index=False,encoding='UTF-8')